In [1]:
from studies.study_factory import StudyFactory
from dataloader.dataloader import DataLoader

dataloader = DataLoader(
    buffer_size=10,
    max_cache_size_gb=100,
    cache_dir="cache",
    notch_filter=True,
    frequency_bands={"all": (0.5, 100)},
    scaling="both",
    brain_clipping=20,
    baseline_window=0.5,
    new_freq=100,
    batch_types={"audio": 10},
    batch_kwargs={
        'audio': {
            'max_random_shift': 1,
            'window_size': 4,
            'window_stride': 1, 
            'audio_sample_rate': 16000,
            'hop_length': 160,
            'audio_processor': "openai/whisper-large-v3"
        }
    },
)

2024-12-10 04:23:04,895	INFO worker.py:1821 -- Started a local Ray instance.


In [2]:
study = StudyFactory.create_study(
    study_name='gwilliams2023',
    batch_type='audio',
    path='data/gwilliams2023',
    cache_enabled=True,
    max_cache_size=200, # in items
)

Loading GWilliams2023 with batch type audio


In [3]:
recordings = []

# Unfold all recordings (3 dim) of python list to 1
from itertools import chain
import random

flat_recordings = list(chain.from_iterable(chain.from_iterable(study.recordings)))
# random.shuffle(flat_recordings)

In [4]:
# # Start background fetching
import time


dataloader.start_fetching(flat_recordings, cache=True)

# Process batches as they become available
try:
    batches, recs, start_time = 0, 0, time.time()
    print(f'Total recordings: {len(flat_recordings)}')
    
    while True:
        batch = dataloader.get_recording()
        
        if batch is None:
            break
        
        brain = batch.brain_segments['all']
        batches += brain.shape[0]
        recs += 1
        
        if recs % 10 == 0:
            print(f"Batch {batches} ({recs} recordings) processed in {time.time() - start_time:.2f}s")
            print(
                f"Average processing time per recording: {(time.time() - start_time) / recs:.2f}s"
            )
            print(
                f"Average processing time per batch: {(time.time() - start_time) / batches:.2f}s"
            )
        
except KeyboardInterrupt:
    print("Interrupted")
    dataloader.stop()
except Exception as e:
    print("Error", e)
    dataloader.stop()

Total recordings: 196
Batch 3721 (10 recordings) processed in 6.22s
Average processing time per recording: 0.62s
Average processing time per batch: 0.00s
Batch 10709 (20 recordings) processed in 10.36s
Average processing time per recording: 0.52s
Average processing time per batch: 0.00s
Batch 16519 (30 recordings) processed in 13.75s
Average processing time per recording: 0.46s
Average processing time per batch: 0.00s
Batch 21638 (40 recordings) processed in 18.93s
Average processing time per recording: 0.47s
Average processing time per batch: 0.00s
Batch 28654 (50 recordings) processed in 122.79s
Average processing time per recording: 2.46s
Average processing time per batch: 0.00s
Batch 34823 (60 recordings) processed in 127.36s
Average processing time per recording: 2.12s
Average processing time per batch: 0.00s
Batch 41194 (70 recordings) processed in 132.07s
Average processing time per recording: 1.89s
Average processing time per batch: 0.00s
Batch 46019 (80 recordings) processed i

(raylet) Spilled 2854 MiB, 5 objects, write throughput 1369 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 5125 MiB, 11 objects, write throughput 1658 MiB/s.
(raylet) Spilled 9288 MiB, 18 objects, write throughput 1684 MiB/s.
(raylet) Spilled 17407 MiB, 37 objects, write throughput 1735 MiB/s.
